In [1]:
from copy import deepcopy
from collections import defaultdict
import random
import numpy as np
import time 
import json

## Reading and initializing functions

In [2]:
def get_sudokus(filename, sudokusize=9):
    '''
    get_states is to read multiple sudokus in the format:
    ..8..4..5...64..5 etc.
    returns a list of sudoku board states as clauses in a dict
    '''
    sudokus = []
    with open(filename) as sudoku_states:
        for i, state in enumerate(sudoku_states):
            sudoku = {}
            pointers = {}
            row = 1
            index = 0
            for j, content in enumerate(state):
                if j%sudokusize == 0 and j > 0:
                    row+=1
                try:
                    int(content)
                    literal = int(str(row) + str(j%sudokusize+1) + content)
                    sudoku[index] = [literal]
                    if literal not in pointers.keys():
                        pointers[literal] = [index]
                    else:
                        pointers[literal].append(index)
                    index += 1
                except:
                    # is not convertible to int
                    continue
            sudokus.append((sudoku, pointers))
    return sudokus

def get_constraints_from_dimacs(filename, prevConstraints={}, prevPointers={}):
    '''
    Convert dimacs file to a dictionary of constraints
    and pointers. Allows merging with previous pointers and constraints.
    '''
    constraints = prevConstraints # dictionary with constraints as: clausenumber->[literals]
    pointers = prevPointers # dictionary with pointers to clause numbers for literals as: literals->[clausenumbers]
    initial_length = len(constraints)
    with open(filename) as f:
        for i, clause in enumerate(f):
            index = i + initial_length
            if clause[0] == 'c' or clause[0] == 'p':
                # skip comments etc.
                continue
            constraints[index]= []
            for literal in clause.split():
                literal = int(literal)
                if literal != 0:
                    constraints[index].append(literal)
                    if literal not in pointers.keys():
                        pointers[literal] = [index]
                    else:
                        pointers[literal].append(index)
                    
    return constraints, pointers

    

## DP helper functions for constraints updates

In [ ]:
def assign_literal(constraints, pointers, literal, assignments, random=False):
    '''
    assign the given literal to be True.
    assign_literal is aimed to be one recursion stack, so deepcopies are made
    in order to properly backtrack
    '''
    constraintsCopied = deepcopy(constraints)
    pointersCopied = deepcopy(pointers)
    assignmentsCopied = deepcopy(assignments)
    assignmentsUpdated = update_assigments(assignmentsCopied, literal)
    constraintsUpdated, pointersUpdated = update_constraints(constraintsCopied, pointersCopied, literal)
    if random:
        extract_features(constraints, constraintsUpdated, pointers, pointersUpdated, assignments, literal)
    return constraintsUpdated, pointersUpdated, assignmentsUpdated
     
def update_constraints(constraints, pointers, literal):
    '''
    literal: e.g. 112 or 112'
    Updates constraints and pointers by removing all clauses 
    with the given literal, because the clause becomes true.
    Remove counter-literal out of all clauses.
    '''
    ## Find clauses with literal and remove them

    for lit in literal:
        clauses=list(pointers[lit])
        for clause in list(pointers[lit]):
            for l in constraints[clause]:
                pointers[l].remove(clause)
#             print(clause)
#             print(literal)
#             print(pointers[lit])
            del constraints[clause]
        del pointers[lit]
    
    ## Find clauses with counter-literal and remove its counter-literal

    for lit in literal:
        counterLiteral = -lit
        if counterLiteral in pointers.keys():
            counters=list(pointers[counterLiteral])
            for clause in counters:
                constraints[clause].remove(counterLiteral)
            del pointers[counterLiteral]
        else:
            print('no counter lit')
        
    
#     for clause in clauses:
#         constraints[clause].remove(counterLiteral)
   
    return constraints, pointers

def update_assigments(assignments, literal):
    '''
    Keep track of assigned literals.
    Needed in order to return satisfiable solution of literals.
    '''
    for lit in literal:
        if lit in assignments:
            print("already assigned", lit, "(in this stack), returning...")
#             return
        else:
            assignments.append(lit)
    return assignments
    
def check_tautologies(constraints, pointers):
    '''
    Check for clauses with a tautology and remove
    them.
    '''
    foundOne = False
    for clause in constraints.keys():
        literals = constraints[clause]
        counterLiterals = [-x for x in literals]
        for i in literals:
            for j in counterLiterals:
                if i == j:
                    del constraints[clause]
                    pointers[i].remove(clause)
                    foundOne = True
                    break
    if foundOne:
        print("Found tautologies and removed corresponding clauses")
    else:
        print("No tautologies found")
                  
    return constraints, pointers  

### Extract features to use as regressor datapoints

In [4]:
def extract_features(constraintsOld, constraintsNew, pointersOld, pointersNew, assignments, literals):
    for literal in literals:
        features[literal] = {}

        ## Ratio of the polarity of this literal before assignment
        positive = len(pointersOld[literal])
        negative = len(pointersOld[-literal])
        ratio = positive/(positive+negative)
        features[literal]["posneg_ratio"] = ratio

        ## Clauses that were resolved with this assignment
        clausesResolved = len(constraintsOld) - len(constraintsNew)
        features[literal]["c_res"] = clausesResolved

        ## Clauses that became empty with this assignment
        emptyClauses = 0
        for clause in constraintsNew.keys():
            if not constraintsNew[clause]:
                emptyClauses += 1
        features[literal]["empty_c"] = emptyClauses

        ## Average clause length with given literal before assignment
        clauses = pointersOld[literal]
        lengths = [len(constraintsOld[c]) for c in clauses]
        features[literal]["c_len"] = np.mean(lengths) 

        ## Label whether this assignment results in succes (to be determined at halt)
        features[literal]["SAT"] = 0
    

## DP cases

In [22]:
def empty_clause(constraints):
    for clause in constraints.keys():
        if not constraints[clause]:
            return True
        
    return False

def unit_clause(constraints): 
    units=[]
    for clause in constraints.keys():
        if len(constraints[clause])==1:
            literal = next(iter(constraints[clause]))
            units.append(literal)

    return list(set(units))

def unit_propagate(constraints, pointers, assignments):
    '''
    Automatically assign the literal of all unit clauses
    until there are no unit clauses left.
    '''
    literals = unit_clause(constraints)
    constraints, pointers, assignments = assign_literal(constraints, pointers, literals, assignments)
#     while(not empty_clause(constraints) and literal):
#         print("found unit clause")
#         constraints, pointers, assignments = assign_literal(constraints, pointers, literal, assignments)
#         literal = unit_clause(constraints)
    return constraints, pointers, assignments

def pure_literal(pointers):
    literals=list(pointers.keys())
    for literal in literals:
        if -literal not in literals:
            return [literal],True
        
    return None,False

def random_literal(pointers):
    literals=list(pointers.keys())
    literal = random.choice(literals) 
#     if len(pointers[literal])==0:
#         print("super weird")
#     print("dit is de random",literal)
    return [literal]

In [6]:
a=[5,6]
a.remove(5)
a

[6]

## DP main function

In [7]:
def DP(constraints, pointers, assignments,split): 
    '''
    DP main recursive function. Always returns satisfiable
    assignment for all constraints if there is one.
    '''
    constraints, pointers, assignments = unit_propagate(constraints, pointers, assignments)
    
    if len(constraints)==0:
        return assignments
   
    if empty_clause(constraints):
        return False
    
    literal,status = pure_literal(pointers)
    if status:
#         print("found literal")
        constraints, pointers, assignments = assign_literal(constraints, pointers, literal, assignments,random=False)
        DP(constraints, pointers, assignments,split)

    if split=="random":
#         print("rand assignment")
        literal = random_literal(pointers)
    if split=="jerow":
        literal=jeroSloWang(constraints,pointers)
    if split=="moms":
        literal=mom(constraints,pointers,3)
#     print(pointers.keys())
#     print("goes fault in true")
    constraintsUpdated, pointersUpdated, assignmentsUpdated = assign_literal(constraints, pointers, literal, assignments, random=True)
#     constraintsUpdated, pointersUpdated, assignmentsUpdated = assign_literal(constraints, pointers, literal, assignments, random=True)

    assignmentsFinal = DP(constraintsUpdated, pointersUpdated, assignmentsUpdated,split)
    if assignmentsFinal:
        return assignmentsFinal
    else:
#         constraints, pointers, assignments = assign_literal(constraints, pointers, -literal, assignments, random=True)
#         print("goes fault in false")
        constraints, pointers, assignments = assign_literal(constraints, pointers, [literal[0]*-1], assignments, random=True)

        return DP(constraints, pointers, assignments,split)

In [14]:
#jeroslow wang heuristic
def jeroSloWang(constraints,pointers):
    j={}
    maxValue=0
    maxLiteral=None
#     print(pointers.keys())
    if len(pointers.keys())==0:
        print('THIS IS NOT GOOD')
    for literal in pointers.keys():
        for clause in pointers[literal]:
            if literal not in j.keys():
                j[literal]=0
            j[literal]+=(2**(-len(constraints[clause])))
        try:
            if j[literal]>maxValue:
                maxValue=j[literal]
                maxLiteral=literal
        except:
            continue
#     print(j)
    return [maxLiteral]
# jeroSloWang(constraints,pointers)   

In [36]:
#Mom's heuristic
import numpy as np
def mom(constraints,pointers,k):
    smallestClasses=[]
    minClass=np.inf
    for clause in constraints.keys():
#         print(minClass)
        if len(constraints[clause])<minClass:
            minClass=len(constraints[clause])
            smallestClasses=[]
            smallestClasses.append(clause)
        elif len(constraints[clause])==minClass:
            smallestClasses.append(clause)
#     print(smallestClasses)
#     litsInSmallestClause=[]
#     for sClass in smallestClasses:
#         for lit in constraints[sClass]:
#             litsInSmallestClause.append(lit)
    value=0
    maxLiteral=None
    haveSeen=[]
    
    for literal in pointers.keys():
        if literal not in haveSeen and -literal not in haveSeen:
            X=len([x for x in pointers[literal] if x in smallestClasses])
            X_=len([x for x in pointers[-literal] if x in smallestClasses])
#             print(X,X_)
            momsValue=(X+X_)*2^k+(X+X_)
            haveSeen.append(literal)
            if momsValue>value:
                value=momsValue
                maxLiteral=literal
#             print(maxLiteral)
#     print(maxLiteral)
    if maxLiteral:
        return [maxLiteral]
    else:
        return []
# mom(constraints,pointers,3)     


### Script

In [37]:
def run():    
    datapoints = []
    
    train_sudokus = [x for x in range(0,1000) if x%2 == 0]
    
    ## Run DP
    typesOfSplit=["moms"]
    sudokus=get_sudokus('1000 sudokus.txt')
    for split in typesOfSplit:
    
        for i in train_sudokus[0:100]:
            ## Get board and rules
            startTime = time.time()
            print("Reading and analyzing...")
            constraints, pointers = sudokus[i]
            constraints, pointers = get_constraints_from_dimacs('sudoku-rules.txt', constraints, pointers)
            constraints, pointers = check_tautologies(constraints, pointers)
            print("Starting DP algorithm...")
            global features
            features = {}
            assignments = DP(constraints, pointers, [],split)
#             print(len(assignments))
            endTime = time.time()
            if assignments:
                print("Solution found")
#                 print(split)
                print("Time to complete by",split,":", endTime-startTime)
                boardPositions = [a for a in assignments if a > 0]
                print(boardPositions)
#                 print(assignments)
                for a in assignments:
                    if a in features.keys():
                        features[a]["SAT"] = 1
                for f in features.keys():
                    datapoints.append(features[f])
            else:
                print("No solution found")
            
    with open('data.json', 'w') as outfile:
        json.dump(datapoints, outfile)
run()

Reading and analyzing...
No tautologies found
Starting DP algorithm...


C:\Users\larsr\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\larsr\Anaconda3\lib\site-packages\numpy\core\_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Solution found
Time to complete by moms : 1.9682934284210205
[129, 642, 897, 134, 774, 392, 523, 532, 936, 171, 428, 686, 943, 183, 451, 964, 868, 357, 998, 366, 755, 117, 226, 196, 927, 212, 876, 746, 416, 556, 793, 767, 162, 841, 261, 145, 158, 253, 548, 678, 654, 663, 473, 637, 447, 514, 824, 482, 972, 494, 722, 852, 959, 249, 284, 295, 344, 388, 587, 238, 277, 379, 575, 569, 465, 718, 591, 439, 819, 699, 731, 789, 313, 833, 915, 885, 981, 321, 611, 335, 625]
Reading and analyzing...
No tautologies found
Starting DP algorithm...
Solution found
Time to complete by moms : 4.336040258407593
[261, 523, 396, 536, 794, 928, 931, 678, 167, 945, 699, 574, 452, 581, 851, 983, 992, 354, 868, 229, 615, 365, 191, 512, 771, 762, 682, 555, 788, 569, 298, 548, 597, 493, 158, 895, 663, 737, 634, 725, 175, 476, 485, 235, 464, 966, 886, 126, 716, 844, 914, 113, 214, 824, 184, 132, 833, 877, 819, 957, 389, 338, 149, 822, 287, 759, 656, 979, 439, 418, 743, 246, 253, 311, 272, 373, 342, 317, 327, 627, 4

Solution found
Time to complete by moms : 2.1750993728637695
[526, 272, 146, 915, 533, 662, 419, 677, 427, 194, 963, 716, 214, 986, 991, 225, 739, 358, 493, 369, 755, 373, 634, 476, 129, 435, 344, 656, 866, 113, 512, 482, 165, 152, 643, 253, 545, 559, 875, 598, 567, 261, 979, 247, 137, 849, 238, 957, 817, 311, 289, 932, 296, 336, 322, 924, 618, 621, 699, 892, 831, 742, 685, 948, 387, 451, 741, 395, 854, 797, 468, 441, 764, 171, 464, 574, 581, 778, 188, 784, 723, 828, 883]
Reading and analyzing...
No tautologies found
Starting DP algorithm...
Solution found
Time to complete by moms : 2.0706095695495605
[654, 527, 534, 919, 279, 922, 156, 287, 672, 816, 946, 691, 442, 971, 332, 731, 864, 225, 488, 365, 628, 635, 767, 613, 512, 647, 715, 823, 411, 357, 724, 117, 293, 236, 133, 343, 426, 439, 329, 381, 463, 121, 455, 968, 845, 749, 838, 937, 953, 559, 169, 851, 258, 899, 548, 262, 877, 752, 689, 666, 561, 192, 144, 241, 882, 497, 214, 474, 318, 185, 394, 178, 984, 376, 798, 575, 995, 586, 

Solution found
Time to complete by moms : 1.7046420574188232
[519, 268, 915, 283, 416, 161, 291, 424, 939, 431, 688, 178, 565, 313, 185, 954, 345, 987, 992, 232, 749, 374, 766, 382, 321, 225, 963, 635, 664, 489, 129, 338, 369, 659, 136, 528, 197, 899, 279, 114, 357, 396, 926, 948, 217, 798, 458, 244, 256, 612, 971, 818, 693, 594, 495, 443, 641, 581, 711, 784, 627, 533, 142, 153, 552, 737, 546, 834, 676, 847, 851, 886, 722, 467, 755, 472, 862, 577, 773, 875, 823]
Reading and analyzing...
No tautologies found
Starting DP algorithm...
Solution found
Time to complete by moms : 2.0037002563476562
[131, 264, 651, 766, 275, 936, 432, 819, 188, 572, 447, 959, 453, 712, 335, 977, 733, 357, 613, 625, 755, 891, 638, 727, 867, 915, 885, 834, 741, 281, 876, 921, 554, 944, 828, 361, 843, 471, 511, 258, 968, 852, 163, 778, 537, 156, 548, 318, 498, 239, 145, 687, 373, 662, 646, 465, 595, 242, 349, 223, 993, 583, 569, 429, 526, 486, 982, 216, 297, 117, 789, 414, 384, 396, 179, 322, 674, 794, 124, 192, 

Solution found
Time to complete by moms : 2.0177388191223145
[266, 783, 912, 273, 539, 163, 554, 185, 572, 959, 448, 834, 197, 332, 591, 469, 217, 235, 364, 626, 118, 887, 633, 282, 515, 422, 543, 493, 528, 437, 456, 586, 853, 567, 923, 677, 313, 484, 947, 475, 727, 411, 136, 614, 744, 825, 357, 258, 846, 345, 819, 716, 121, 152, 224, 329, 174, 241, 149, 642, 689, 994, 299, 779, 698, 396, 965, 976, 795, 655, 661, 892, 751, 762, 738, 931, 868, 871, 378, 988, 381]
Reading and analyzing...
No tautologies found
Starting DP algorithm...
Solution found
Time to complete by moms : 4.472921371459961
[641, 275, 918, 539, 672, 161, 547, 683, 825, 837, 326, 844, 973, 591, 211, 469, 992, 866, 614, 359, 238, 374, 254, 699, 564, 381, 627, 898, 889, 229, 871, 179, 719, 949, 188, 282, 134, 115, 123, 317, 393, 332, 762, 263, 853, 812, 157, 967, 297, 246, 196, 513, 142, 443, 733, 936, 416, 431, 635, 656, 668, 365, 555, 452, 745, 758, 348, 586, 522, 428, 951, 985, 578, 495, 721, 794, 477, 776, 787, 924, 4

Solution found
Time to complete by moms : 3.527813673019409
[134, 775, 649, 784, 914, 148, 286, 418, 167, 551, 816, 691, 436, 193, 961, 583, 329, 211, 855, 492, 625, 378, 766, 421, 935, 115, 341, 126, 546, 356, 524, 731, 676, 996, 474, 489, 385, 577, 595, 568, 688, 152, 179, 871, 181, 662, 272, 959, 973, 654, 928, 269, 758, 253, 227, 898, 457, 364, 238, 465, 245, 294, 397, 839, 844, 799, 443, 863, 532, 519, 717, 723, 822, 887, 637, 333, 312, 613, 947, 982, 742]
Reading and analyzing...
No tautologies found
Starting DP algorithm...
Solution found
Time to complete by moms : 2.4708478450775146
[398, 796, 156, 548, 679, 171, 563, 947, 438, 311, 187, 572, 834, 721, 855, 862, 616, 622, 495, 252, 245, 124, 637, 586, 261, 688, 531, 665, 442, 168, 846, 466, 693, 367, 481, 641, 951, 891, 654, 758, 344, 353, 743, 597, 514, 457, 559, 149, 474, 525, 192, 115, 332, 133, 329, 385, 423, 419, 236, 376, 926, 273, 889, 299, 284, 994, 813, 782, 983, 764, 717, 912, 227, 739, 969, 218, 828, 935, 775, 877, 9

Solution found
Time to complete by moms : 1.3278403282165527
[515, 643, 396, 527, 656, 914, 538, 158, 289, 549, 946, 821, 322, 581, 972, 339, 211, 341, 353, 865, 879, 883, 632, 764, 367, 576, 963, 169, 619, 431, 562, 661, 894, 266, 318, 554, 624, 593, 786, 499, 468, 848, 482, 113, 474, 416, 225, 423, 273, 733, 384, 375, 145, 836, 935, 298, 455, 252, 126, 134, 795, 187, 447, 192, 244, 237, 742, 812, 685, 857, 678, 171, 697, 988, 991, 929, 997, 717, 728, 759, 777, 751, 959]
Reading and analyzing...
No tautologies found
Starting DP algorithm...
Solution found
Time to complete by moms : 1.408357858657837
[394, 922, 288, 674, 554, 427, 687, 569, 699, 191, 831, 964, 329, 458, 211, 341, 739, 868, 996, 623, 753, 377, 766, 255, 951, 419, 547, 521, 148, 661, 434, 244, 728, 857, 465, 226, 279, 797, 159, 443, 978, 598, 816, 356, 646, 492, 895, 293, 652, 125, 536, 824, 512, 385, 114, 575, 715, 784, 583, 873, 618, 917, 635, 481, 771, 742, 945, 338, 313, 163, 933, 989, 362, 172, 849, 882, 186, 476, 1

Solution found
Time to complete by moms : 2.1158831119537354
[385, 774, 136, 528, 273, 149, 667, 939, 941, 563, 826, 587, 718, 464, 592, 211, 735, 871, 237, 493, 757, 631, 766, 255, 432, 481, 869, 551, 721, 338, 534, 347, 799, 965, 833, 743, 895, 817, 782, 983, 427, 888, 184, 958, 694, 112, 125, 153, 914, 372, 922, 262, 361, 289, 168, 177, 224, 298, 396, 686, 191, 354, 997, 844, 976, 246, 456, 545, 642, 516, 852, 579, 659, 448, 678, 329, 419, 623, 313, 475, 615]
Reading and analyzing...
No tautologies found
Starting DP algorithm...
Solution found
Time to complete by moms : 3.3927531242370605
[768, 259, 392, 527, 146, 531, 664, 926, 416, 422, 687, 945, 818, 691, 184, 953, 321, 333, 225, 358, 873, 619, 241, 498, 344, 548, 171, 236, 317, 214, 112, 365, 262, 911, 434, 157, 195, 163, 772, 932, 582, 277, 293, 288, 978, 513, 483, 994, 643, 635, 715, 723, 628, 989, 574, 129, 138, 386, 676, 599, 967, 842, 555, 652, 754, 475, 885, 824, 379, 781, 447, 451, 749, 469, 566, 856, 796, 737, 839, 861, 